In [1]:
from __future__ import print_function

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, GlobalMaxPooling2D, Lambda, UpSampling2D, BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import EfficientNetB0

import matplotlib.pyplot as plt
import numpy as np

In [2]:
print('tensorflow:', tf.__version__)
print('keras:', tensorflow.keras.__version__)

##Uncomment the following two lines if you get CUDNN_STATUS_INTERNAL_ERROR initialization errors.
## (it happens on RTX 2060 on room 104/moneo or room 204/lautrec) 
##physical_devices = tf.config.experimental.list_physical_devices('GPU')
##tf.config.experimental.set_memory_growth(physical_devices[0], True)

tensorflow: 2.4.1
keras: 2.4.0


In [3]:
#load (first download if necessary) the CIFAR10 dataset
# data is already split in train and test datasets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')


x_train = (x_train / 255) - 0.5
x_test = (x_test / 255) - 0.5

# One hot encode
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

170500096/170498071 [==============================] - 96s 1us/step


In [5]:
####--------  PARAMETERS --------####

model_name = "resnet_frozen_layers"
results_path = "results/transfer_learning"

learning_rate = 0.001
num_epochs = 10
val_split = 0.2
batch = 64
input_shape = x_train.shape[1:]
num_classes = y_train.shape[1]
num_samples = y_train.shape[0]

In [6]:
####-------- PROCESS DATA ACCORDING TO BACKBONE --------####

x_train = tf.keras.applications.efficientnet.preprocess_input(x_train)
x_test = tf.keras.applications.efficientnet.preprocess_input(x_test)

In [7]:
####-------- MODEL DEFINITION --------####

conv_base = EfficientNetB0(weights='imagenet', include_top=False,
                           pooling='avg', input_shape=(256,256,3))

for layer in conv_base.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False

model = Sequential()
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(conv_base)
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dense(10, activation="softmax"))

opt = Adam(learning_rate = learning_rate)
model.compile(optimizer = opt, loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

16711680/16705208 [==============================] - 4s 0us/step


In [8]:
####-------- TRAINING --------####

earlyStopping = EarlyStopping(monitor='val_loss',
                              patience=20,
                              verbose=0,
                              mode='min',
                              restore_best_weights=True)

history = model.fit(x_train, y_train, validation_split = val_split,
                    epochs = num_epochs, callbacks =[earlyStopping],
                    batch_size = batch)

Epoch 1/10
625/625 [==============================] - 128s 197ms/step - loss: 1.0805 - accuracy: 0.6416 - val_loss: 1.2948 - val_accuracy: 0.5680
Epoch 2/10
625/625 [==============================] - 122s 195ms/step - loss: 0.3917 - accuracy: 0.8656 - val_loss: 0.2556 - val_accuracy: 0.9108
Epoch 3/10
625/625 [==============================] - 122s 196ms/step - loss: 0.2747 - accuracy: 0.9043 - val_loss: 0.2213 - val_accuracy: 0.9222
Epoch 4/10
625/625 [==============================] - 122s 196ms/step - loss: 0.2188 - accuracy: 0.9269 - val_loss: 0.2190 - val_accuracy: 0.9244
Epoch 5/10
625/625 [==============================] - 122s 196ms/step - loss: 0.1674 - accuracy: 0.9421 - val_loss: 0.1859 - val_accuracy: 0.9389
Epoch 6/10
625/625 [==============================] - 122s 196ms/step - loss: 0.1418 - accuracy: 0.9491 - val_loss: 0.1827 - val_accuracy: 0.9425
Epoch 7/10
625/625 [==============================] - 122s 196ms/step - loss: 0.1293 - accuracy: 0.9563 - val_loss: 0.1982 -

In [9]:
####-------- EVALUATION ACCURACY --------####

_, train_acc = model.evaluate(x_train, y_train, verbose=0)
_, test_acc = model.evaluate(x_test, y_test, verbose=0)

print(train_acc, test_acc)

0.9815800189971924 0.9387999773025513
